In [6]:
import pandas as pd
import numpy as np

In [7]:
from nltk import tokenize
from operator import itemgetter
import math
from tqdm import tqdm
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))

In [8]:
kk = os.listdir("/home/amansinha/extractpubmed/papers/key_meta_all/")
kk

['tdm1_metadf2.csv',
 'genome wide association study_metadf2.csv',
 'progression free survival_metadf2.csv',
 'hormone therapy_metadf2.csv',
 'overall survival_metadf2.csv',
 'disease free survival_metadf2.csv',
 'radiation_metadf2.csv',
 'chemotherapy_metadf2.csv',
 'neratinib_metadf2.csv',
 'pertuzumab_metadf2.csv',
 'pathologic complete response_metadf2.csv',
 'surgery_metadf2.csv',
 'polymorphism_metadf2.csv',
 'neoadjuvant_metadf2.csv',
 'adjuvant_metadf2.csv',
 'trastuzumab_metadf2.csv',
 'polygenic risk score_metadf2.csv',
 'Breast Cancer_metadf2.csv']

In [66]:
def get_keywords(doc, k):
    total_words = doc.split()
    total_word_length = len(total_words)
    #print(total_word_length)
    
    total_sentences = tokenize.sent_tokenize(doc)
    total_sent_len = len(total_sentences)
    #print(total_sent_len)
    
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.', '')
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1
            
    tf_score.update((x, y/int(total_word_length)) for x,y in tf_score.items())
    #print(tf_score)
    
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))
    
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in stop_words:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)
    
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    
    result = dict(sorted(tf_idf_score.items(), key = itemgetter(1), reverse = True)[:k]) 
    return result, tf_idf_score
    
    

In [41]:
df = pd.read_csv('/home/amansinha/extractpubmed/papers/key_meta_all/tdm1_metadf2.csv')
df['keywords'] = df['keywords'].str.strip('[]').str.split(',')
df['keywords'] = df['keywords'].apply(lambda x: [xx.strip().strip("''") for xx in x])
df

,pubmed_id,title,keywords,publication_date,abstract
0,33479246,Fibroblast growth factor receptor facilitates ...,[],2021-01-23,Trastuzumab-emtansine (T-DM1) is an antibody-d...
1,33259892,Third-line treatment of HER2-positive advanced...,"[Breast cancer, HER2\xa0+, Third line, Trastuz...",2020-12-02,Human epidermal growth factor receptor 2 posit...
2,33238772,Margetuximab for the treatment of HER2-positiv...,"[Margetuximab, breast cancer, her2, mgah22, so...",2020-11-27,No specific standard treatment is currently re...
3,32737515,Post-neoadjuvant treatment with capecitabine a...,"[Breast cancer, Capecitabine, Radiochemotherap...",2020-08-02,Following neoadjuvant chemotherapy for breast ...
4,32236828,Comparison of outcomes in a population-based c...,"[Cardiotoxicity, Heart failure, Overall surviv...",2020-04-03,Little data exist for comparing cardiac safety...
5,32192389,Safety of trastuzumab after trastuzumab emtans...,"[Breast cancer, HER2, nodular regenerative hyp...",2020-03-21,Trastuzumab emtansine is an antibody-drug conj...
6,32104023,Analysis of the Cost-Effectiveness of Liquid B...,"[biomarkers, breast cancer, cost-effectiveness...",2020-02-28,Breast cancer is highly prevalent worldwide an...
7,31949426,Recent advances and optimal management of huma...,"[Breast cancer, TDM1, neratinib, pertuzumab, t...",2020-01-18,With the introduction of anthracycline-based r...
8,31781874,When to Add Additional Anti-HER2 Therapy to Ad...,"[Adjuvant, Early, HER2, Neoadjuvant, Neratinib...",2019-11-30,One year of trastuzumab dramatically improves ...
9,31449983,The evolving role of trastuzumab emtansine (T-...,"[Brain metastases, Breast cancer, HER2, Trastu...",2019-08-27,Approximately 30-50% of advanced human epiderm...


In [ ]:
columns = [f'key{i+1}' for i in range(10)]

In [88]:
for k in kk:
    
    df = pd.read_csv(f'/home/amansinha/extractpubmed/papers/key_meta_all/{k}')
    df['keywords'] = df['keywords'].str.strip('[]').str.split(',')
    df['keywords'] = df['keywords'].apply(lambda x: [xx.strip().strip("''") for xx in x])
    
    
    name = k.split('_')[0]

    akeys, amkeysdict, all_text = set(),[], []
    for t in ['title', 'abstract']:
        text = []
        for i, (key, abst) in enumerate(zip(df['keywords'], df[t])):
            if len(key)>1  and (not pd.isna(abst)):
                    keylist = set(list(map(str.lower, key)))

                    amkeysdict.append(d)
                    text.append(abst)
                    akeys |= keylist
            if i == 100:
                break
                
        all_text.append(' '.join(text))
        mkd,d =  get_keywords(' '.join(all_text), 10)
        manual_keylist = set(list(map(str.lower,mkd.keys() )))
        
        print(f'{name}- {manual_keylist}')
        
    #print(f'\n all_keys: {akeys}')
        
    print('*'*30)
    print('\n')

tdm1- {'post-neoadjuvant', 'her2-positive', 'breast', 'cancer', 't-dm1', 'trastuzumab', 'cancer:', 'a', 'safety'}
tdm1- {'her2-positive', 'her2', 'we', 'trastuzumab', 'the', 'tdm1', 'egdr', 'trastuzumab/pertuzumab', '=', '<'}
******************************


genome wide association study- {'mendelian', 'genome-wide', "parkinson's", 'randomization', 'risk', 'with', 'variants', 'study', 'genetic', 'gwas'}
genome wide association study- {'ci:', 'genome-wide', 'the', '95%', '(gwas)', 'in', 'risk', '=', 'we', 'gwas'}
******************************


progression free survival- {'neck', 'patients', 'neoadjuvant', 'efficacy', 'squamous', 'lung', 'with', 'a', 'study', 'without'}
progression free survival- {'(pfs)', 'the', '<', 'pfs', 'nsclc', 'os', '=', '±', 'survival', 'we'}
******************************


hormone therapy- {'breast', 'cancer', 'growth', 'the', 'review', 'hormone', 'with', 'a', 'therapy'}
hormone therapy- {'abi', 'dna', 'the', '95%', 'pfs', 'in', '=', 'a', 'we', 'therapy'}
***

In [87]:
for k in kk:
    
    df = pd.read_csv(f'/home/amansinha/extractpubmed/papers/key_meta_all/{k}')
    df['keywords'] = df['keywords'].str.strip('[]').str.split(',')
    df['keywords'] = df['keywords'].apply(lambda x: [xx.strip().strip("''") for xx in x])
    
    
    name = k.split('_')[0]

    akeys, amkeysdict, all_text = set(),[], []
    for t in ['title', 'abstract']:
        text = []
        for i, (key, abst) in enumerate(zip(df['keywords'], df[t])):
            if len(key)>1  and (not pd.isna(abst)):
                    keylist = set(list(map(str.lower, key)))

                    amkeysdict.append(d)
                    text.append(abst)
                    akeys |= keylist
            if i == 100:
                break
                
        all_text.append(' '.join(text))
        mkd,d =  get_keywords(' '.join(all_text), 10)
        manual_keylist = set(list(map(str.lower,mkd.keys() )))
        
        print(f'{name}- {manual_keylist}')
        
    print(f'\n all_keys: {akeys}')
        
    print('*'*30)
    print('\n')
            
        
    #break
        
                
            
            



tdm1- {'post-neoadjuvant', 'her2-positive', 'breast', 'cancer', 't-dm1', 'trastuzumab', 'cancer:', 'a', 'safety'}
tdm1- {'her2-positive', 'her2', 'we', 'trastuzumab', 'the', 'tdm1', 'egdr', 'trastuzumab/pertuzumab', '=', '<'}

 all_keys: {'post-neoadjuvant', 'her2-positive', 'her2-positive disease', 'neratinib', 'surgery', 'targeted therapies', 'cost-effectiveness', 'type 1 diabetes', 'trastuzumab', 'metastatic breast cancer', 'trastuzumab emtansine', 'dairy cow', 'mgah22', 'immunotherapy', 'trastuzumab-dm1', 'her2+ breast cancer', 'tucatinib', 'residual disease', 'tkis', 'biomarkers', 'ranolazine', 'immune microenvironment', 'combination systemic therapy', 'revascularization', 'toxicity', 'herceptin', 'highlights', 'hsp90', 'pathologic complete response', 'metastatic her2\\u2009+\\u2009breast cancer', 'emtansine', 'radiotherapy', 'treatment', 'heart failure', 'insulin resistance', 'trastuzumab deruxtecan', 'isolated cabg', 'brain metastases', 'concurrent treatment', 'capecitabine', 'b

progression free survival- {'neck', 'patients', 'neoadjuvant', 'efficacy', 'squamous', 'lung', 'with', 'a', 'study', 'without'}
progression free survival- {'(pfs)', 'the', '<', 'pfs', 'nsclc', 'os', '=', '±', 'survival', 'we'}

 all_keys: {'osimertinib', 'nonnegative matrix factorization', 'egfr', 'endocrine toxicity', 'sdhb', 'pathological complete response', 'pan-cancer', 'non-randomized confirmatory trial', 'chemoradiotherapy', 'renal cell carcinoma', 'bone metastasis', 'pemetrexed', 'systematic review', 'cancer of unknown primary (cup)', 'anti pd-1', 'sox12', 'immunosuppression status', 'tumor-infiltrating macrophages', 'small cell lung cancer', 'sarcopenic obesity', 'lung adenocarcinoma', 'real-world effectiveness', 'japan', 'immunomodulatory activity', 'clinical manifestations', 'soft tissue sarcoma', 'enzalutamide', 'dyskerin pseudouridine synthase 1', 'pharmacokinetics', 'radiotherapy', 'watchful waiting', 'urothelial carcinoma of the upper urinary tract', 'rs671', 'cancer stem

overall survival- {'the', 'sars-cov-2', '95%', 'que', 'risk', 'covid-19', '=', '±', 'major', 'we'}

 all_keys: {'microbubble', ' grade', 'older adults', 'neurotoxicity', 'chronic kidney disease', 'diabetic foot', 'remember-know procedure', 'aterectomía', 'negative', 'nosocomial infections', 'favipiravir', 'household crowding', 'immune system', 'culture', 'porcine retina', 'rock2', 'edwardsiella piscicida', 'fontan', 'myeloid-derived suppressor cells', 'nuclear factor kappa b', 'pregnancy outcome', 'mammograms', 'multiple pregnancy', 'ez-albi score', 'mortalidad', 'streptococcus pyogenes', 'brentuximab vedotin', 'mortalidad en amputación mayor', 'near-infrared spectroscopy', 'qualitative', 'falls', 'salmonids', 'virulence', 'acute myocardial infarction', 'liver resection', 'breast cancer screening', 'bleeding', 'innate immunity', 'survival analysis', 'co-infection', 'restrictions', 'secondary hyperparathyroidism', 'pim1', 'social isolation', 'disease management', 'atherectomy', 'coronar

radiation- {'dna', 'the', 'qa', 'imrt', 'vmat', 'in', 'circ-ptk2', '=', '5-ala', 'a'}

 all_keys: {'gene set enrichment analysis', 'financial planning', 'financial toxicity', 'mcnp', 'flash', 'pixelated semiconductor', 'risk analysis', 'coordination', 'endocrine toxicity', 'cyanobacteria', 'tumor microenvironment', 'conversion efficiency', 'radiation monitoring', 'hypertrophic olivary degeneration', 'paraneoplatic pemphigus', 'chemoradiotherapy', 'catalysis', 'double strand breaks (dsbs)', 'chemoradiation therapy', 'accelerator', 'dual-ended readout', 'rectal neoplasms', 'anastrazole', 'proton detection', 'anti pd-1', 'flattening filter-free (fff) beams', 'electron beam', 'thermophotovoltaics', 'recurrence', 'objective functions', 'critical-sized bone defects', 'pdsapi', 'hydrogel spacer', 'subglottic small cell carcinoma', 'immune checkpoint inhibitors', 'single-strand annealing', 'ultraviolet radiation', 'caudate lobe', 'picket fence test', 'vascular endothelial growth factor recepto

neratinib- {'her2+', 'her2-positive', 't-dm1', 'trastuzumab', 'the', 'adjuvant', '+', 'in', '=', 'we'}

 all_keys: {'hr+/her2+', 'covalent binding', 'temperature effect on protein binding', 'egfr', 'c-met', 'advanced breast cancer', 'cdk4/6 inhibitors', 'her2 low expression', 'spontaneous metastasis', 'free fraction', 'competition', 'trastuzumab', 'antibody-drug conjugates.', 'drug discovery', 'mouse array', 'second-line treatment', 'immune therapy', 'tucatinib', 'unstable drugs', 'diarrhea', 'extended adjuvant treatment', 'plasma protein binding', 'hormone receptor', 'protein kinase structure', 'neo-antigen', 'michael acceptor', 'toxicity', 'hydrophobic interaction', 'quinoline', 'lobular', 'central nervous system neoplasms', 'targeted therapy', 'metastasis', 'trastuzumab deruxtecan', 'cyclin d1', 'bispecific antibodies', 'breast cancer', 'her2 positive', 'covalent inhibitors', 'intact rb', 'cxcl8', 'human epidermal growth factor receptor 2', 'cholangiocarcinoma', 'cervical cancer', '

pathologic complete response- {'breast', 'her2-positive', 'neoadjuvant', '1', 'mri', 'cancer:', 'with', 'a'}
pathologic complete response- {'however,', 'her2-positive', 'the', '<', '95%', 'neoadjuvant', 'in', '=', 'we'}

 all_keys: {'pathologic response', 'quality of life (qol)', 'neoadjuvant chemotherapy', 'nigeria', 'restricted mean survival times', 'serous carcinoma', 'pathological complete response', 'pathologic complete response (pcr)', 'neoadjuvant treatment', 'chemoradiotherapy', 'kit d816v', 'systematic review', 'rectal neoplasms', 'major pathological response', 'tumor regression grade', 'residual tumour', 'recurrence', 'her2+ breast cancer', 'prostatectomy', 'neoadjuvant immunotherapy', 'triple-negative breast cancer (tnbc)', 'bone lymphoma', 'car-t cells', 'sample size', 'completepathologicalresponse', 'neoadjuvanttherapyinbreastcancer', 'trial design', 'qualitative', 'urothelial carcinoma', 'nduction chemotherapy', 'esmo congress 2020', 'pancreatic neoplasms', 'mir-378a-3p',

polymorphism- {'however,', 'the', 'sars-cov-2', '95%', 'in', 'risk', 'covid-19', '=', 'snp', 'we'}

 all_keys: {'chrysophrys auratus', 'in-situ analytics', 'growth hormone', 'associational effects', 'chronic kidney disease', 'homocysteine', 'hemophagocytic lymphohistiocytosis', 'raman spectroscopy', 'susceptibility', 'atlastin-1', 'cyp19a1', 'genetic variation', 'selective pressure', 'mastitis', 'swine leukocyte antigen', 'supersaturation', 'snp', 'near-infrared spectroscopy', 'world war ii', 'enzymatic activity', 'fat mass', 'ndm-1', 'glyceroneogenesis', 'amomum tsao-ko', 'posttranscription regulation', 'rs671', 'obesity-associated gene protein', 'polymorphic variants', 'cooperation', 'phenotype', 'cancer stroma', 'poorly water-soluble drugs', 'energy framework', 'otc gene', 'plasmids', 'fruit flavor', 'pea', 'hypotension', 'sequence-specific primers pcr', 'epithelial ovarian cancer', 'supra\\xadmolecular framework', 'art', 'vo2max test', 'pyrazine', 'tuberculosis', 'theoretical model

neoadjuvant- {'pd-l1', 'the', '<', '95%', 'neoadjuvant', 'os', '=', '(hr', 'we'}

 all_keys: {'sphincter-saving surgery', 'neoadjuvant chemotherapy', 'intermittent', 'pathological complete response', 'endocrine therapy', 'collusion', 'tumor microenvironment', 'lymph nodes', 'borderline resectable', 'neoadjuvant treatment', 'conversion surgery', 'esophageal neoplasms', 'chemoradiotherapy', 'pemphigoïde bulleuse', 'renal cell carcinoma', 'signet cell adenocarcinoma', 'presentations: presented as a virtual poster presentation at the society of surgical oncology annual cancer symposium', 'multi-vendor', 'peritoneal staging', 'rectal neoplasms', 'placental site trophoblastic tumor', 'sarcome', 'enhanced recovery after surgery', 'locally advanced pancreatic cancer', 'radiodermite', 'her2+ breast cancer', 'pd-1 pathway', 'abdominoperineal resection', 'thyroid autoimmunity', 'immune checkpoint inhibitors', 'compliance', 'urothelial carcinoma', 'august 17-20th', 'pancreatic ductal', 'major hepa

trastuzumab- {'her2-positive', 't-dm1', 'her2', 'the', 'trastuzumab', '(week', 't-dxd', '=', 'we'}

 all_keys: {'her2-targeted therapy', 'selenium (se)', 'surrogate subtyping', 'hr+/her2+', 'dual blockade', 'neoadjuvant chemotherapy', 'somatic mutations', 'antibody drug conjugate (adc)', 'nlr', 'egfr', 'advanced breast cancer', 'endocrine therapy', 'pathological complete response', 'expanded access', 'conversion surgery', 'antibodies', 'neoadjuvant treatment', 'auger electron therapy', 'lyophilization', 'guideline concordance', 'population-based', 'competition', 'trastuzumab', 'adalimumab', 'anti‐inflammatory activity', 'registries', 'bispecific antibody', 'extended adjuvant treatment', 'pcr', 'reduced glutathione (gsh)', 'medicine', 'hydrophobic surface', 'systemic treatment', 'toxicity', 'vinorelbine', 'trastuzumab + pertuzumab', 'her2 amplification', 'salivary duct carcinoma', 'perioperative treatment', 'oncologics', 'a', 'oncology', 'passive adsorption', 'trastuzumab deruxtecan', '

Breast Cancer- {'breast', 'the', 'expression', 'kpna2', '95%', 'in', 'mri', 'wif1', '=', 'hormad1'}

 all_keys: {'pi3k/akt signaling pathway', 'neoadjuvant chemotherapy', 'pixelated semiconductor', 'déficit en testostérone', 'nucleus translocation', 'angiomotin‑like 2', 'advanced breast cancer', 'tumor microenvironment', 'enhancer of zeste homolog\\xa02 polycomb repressive complex subunit\\xa02', 'cancer registries', 'grip strength', 'marital status', 'ovarian clear cell carcinoma (occc)', 'brachial plexus', 'chemoradiotherapy', 'thyrotropin', 'irgd', 'traitement hormonal', 'activity', 'experimental study', 'mammograms', 'mitochondria', 'intracellular ph', 'ovarian', 'recurrence', 'lynch syndrome', 'sympathetic mediated pain', 'rac1', 'biomarker discovery', 'yes-associated protein 1', 'stat3', 'screening mammography', 'neurolysis', 'macrophage', 'risk-reducing surgery', 'thyroid autoimmunity', 'b3 lesions', 'decision aid', 'immune checkpoint inhibitors', 'hormone receptor', 'finite ele

In [69]:
ans, _ = get_keywords(' '.join(all_text), k=10)
ans

{'=': 0.015858450632270954,
 'HER2-positive': 0.008156027624015315,
 'trastuzumab': 0.00802735669239421,
 'We': 0.007689004519923447,
 '<': 0.006317138540394327,
 'eGDR': 0.005317759102235327,
 'TDM1': 0.004644990300307977,
 'The': 0.004398970261438498,
 'HER2': 0.004127846754747094,
 'trastuzumab/pertuzumab': 0.004104793021368067}

In [61]:
global_d = dict()
for dd in amkeysdict:
    global_d.update(**dd)
    
result = dict(sorted(global_d.items(), key = itemgetter(1), reverse = True)[:20]) 
result

{'=': 0.05529651285702302,
 '<': 0.038659677203126394,
 'eGDR': 0.03342207033132504,
 'ISS),': 0.027407525598688507,
 '%': 0.02263337732440638,
 'number': 0.021164799406410542,
 'milk': 0.019185454978530896,
 'sequential': 0.016975032993304783,
 'Conjugates,': 0.016447036378528897,
 '8': 0.01639166043239291,
 'DGAT1': 0.01598787914877575,
 'options,': 0.015873599554807907,
 'effective': 0.015873599554807907,
 'particularly': 0.015873599554807907,
 'Historically,': 0.015873599554807907,
 'considerations': 0.015873599554807907,
 'evaluating': 0.015873599554807907,
 'receptor-positive': 0.015873599554807907,
 'endocrine': 0.015873599554807907,
 'CDK4/6': 0.015873599554807907}

In [59]:
global_d

{'Third-line': 0.0,
 'treatment': 0.009679054179100139,
 'of': 0.0,
 'HER2-positive': 0.015184402281593685,
 'advanced': 0.015184402281593685,
 'breast': 0.0,
 'cancer:': 0.0,
 'From': 0.0,
 'no': 0.0,
 'standard': 0.010268847119406597,
 'to': 0.0,
 'a': 0.0,
 "Pandora's": 0.0,
 'box': 0.0,
 'Margetuximab': 0.010134948692996422,
 'for': 0.0,
 'the': 0.0,
 'metastatic': 0.008223518189264449,
 'cancer': 0.015184402281593685,
 'Post-neoadjuvant': 0.0,
 'with': 0.0,
 'capecitabine': 0.009467281837847648,
 'and': 0.0,
 'trastuzumab': 0.011153424250437662,
 'emtansine': 0.013371339322597426,
 'in': 0.0,
 'patients-sequentially,': 0.0,
 'or': 0.0,
 'better': 0.015184402281593685,
 'simultaneously?': 0.0,
 'Comparison': 0.0,
 'outcomes': 0.009679054179100139,
 'population-based': 0.005689544716191807,
 'cohort': 0.003915330109292958,
 'patients': 0.0025809735750759004,
 'receiving': 0.0,
 'anti-HER2': 0.012890355893727015,
 'therapy': 0.0,
 'clinical': 0.0009193908019709801,
 'trial': 0.001497

In [55]:
get_keywords("Third-line treatment of HER2-positive advanced breast cancer: From no standard to a Pandora's box.", k=10)

{'Third-line': 0.0, 'treatment': 0.0, 'HER2-positive': 0.0, 'advanced': 0.0, 'breast': 0.0, 'cancer:': 0.0, 'From': 0.0, 'standard': 0.0, "Pandora's": 0.0, 'box': 0.0}


({'Third-line': 0.0,
  'treatment': 0.0,
  'of': 0.0,
  'HER2-positive': 0.0,
  'advanced': 0.0,
  'breast': 0.0,
  'cancer:': 0.0,
  'From': 0.0,
  'no': 0.0,
  'standard': 0.0},
 {'Third-line': 0.0,
  'treatment': 0.0,
  'of': 0.0,
  'HER2-positive': 0.0,
  'advanced': 0.0,
  'breast': 0.0,
  'cancer:': 0.0,
  'From': 0.0,
  'no': 0.0,
  'standard': 0.0,
  'to': 0.0,
  'a': 0.0,
  "Pandora's": 0.0,
  'box': 0.0})

In [50]:
result = pd.DataFrame(columns = ['key', 'keywordVtitle_b','keywordVtitle_sbs', 'keywordVtitle_sem', 'keywordVabstract_b','keywordVabstract_sbs', 'keywordVabstract_sem', 'nsamples'])
result['key']  = tn
result['keywordVtitle_b'] = tt1
result['keywordVtitle_sbs'] = tt2
result['keywordVtitle_sem'] = tt3
result['keywordVabstract_b'] = tabst1
result['keywordVabstract_sbs'] = tabst2
result['keywordVabstract_sem'] = tabst3
result['nsamples'] = tc

NameError: name 'tt3' is not defined

In [49]:
result

,key,keywordVtitle_b,keywordVtitle_sbs,keywordVabstract_b,keywordVabstract_sbs,nsamples
0,tdm1,0.112037,0.669167,0.055952,0.377976,28
1,genome wide association study,0.071084,0.761368,0.046135,0.355531,3184
2,progression free survival,0.120741,0.882582,0.067391,0.445179,3607
3,hormone therapy,0.103923,0.853283,0.059205,0.378839,3280
4,overall survival,0.107009,0.880749,0.062405,0.399481,3557
5,disease free survival,0.101470,0.926997,0.055107,0.442631,3551
6,radiation,0.089672,0.881630,0.051175,0.389246,3335
7,chemotherapy,0.107750,0.892227,0.060680,0.414168,3626
8,neratinib,0.128151,0.735046,0.074891,0.383719,201
9,pertuzumab,0.115225,0.787713,0.104231,0.391004,555


Correlation is defined the average of count of intersection of number of keywords between the actual keywords provided by the authors over complete number of samples.

In [7]:
def simple_match(authorkeys, manualkeys):
    return (len(authorkeys & manualkeys)/len(authorkeys))

def substring_match(authorkeys, manualkeys):
    c = 0
    for ak in authorkeys:
        for mk in manualkeys:
            print(ak,mk,int(ak.find(mk)!= -1))
            c += int(ak.find(mk)!= -1)
            '''
            if (ak.find(mk)!= -1):
                if mk not in ak.split(' '):
                    c -= 1
            #print(ak,mk, int(ak.find(mk)!= -1))
            '''
    
    return c/len(authorkeys)
    
def semantic_match(authorkeys, manualkeys, tokenizer, model):
    embdict = dict()
    
    for k in authorkeys | manualkeys:
        inputs = tokenizer.encode_plus(k)
        ids = torch.tensor(inputs['input_ids']).unsqueeze(0)
        mask = torch.tensor(inputs['attention_mask']).unsqueeze(0)
        with torch.no_grad():
            embed = model(ids)[0]
        #print(embed.shape)
        final = torch.mean(embed[:, 1:-1], dim=1).squeeze().detach().numpy()
        
        embdict[k] = final, np.linalg.norm(final)
    
    c= 0
    for ak in authorkeys:
        a = 0
        for mk in manualkeys:
            iak, imk = embdict[ak], embdict[mk]
            a += iak[0].dot(imk[0])/(iak[1]*imk[1])
            #print(ak,mk,  iak[0].dot(imk[0])/(iak[1]*imk[1]))
        c += a/len(manualkeys)
        
        
    
    return c/len(authorkeys)
    


In [2]:
import torch
import numpy as np
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_cased')

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [102]:
tokenizer.tokenize('post-neoadjuvant')

['post', '-', 'neo', '##adjuvant']

In [3]:
#authorkeys =  {'trastuzumab deruxtecan', 'tucatinib', 'ds8201a', 'her2\\xa0+', 'breast cancer', 'third line'} 
#manualkeys = {'from', 'of', 'breast', 'cancer:', 'standard', 'her2-positive', 'third-line', 'no', 'advanced', 'treatment'}

authorkeys= {'t‑dm1', 'radiochemotherapy', 'radiotherapy', 'capecitabine', 'breast cancer'} 
manualkeys= {'with', 'emtansine', 'and', 'trastuzumab', 'post-neoadjuvant', 'treatment', 'in', 'breast', 'cancer', 'capecitabine'}


In [8]:
#print(f'simple: {simple_match(authorkeys, manualkeys)}')
print(f'subs: {substring_match(authorkeys, manualkeys)}')
#print(f'semantic: {semantic_match(authorkeys, manualkeys, tokenizer, model)}')

capecitabine in 1
capecitabine treatment 0
capecitabine capecitabine 1
capecitabine breast 0
capecitabine with 0
capecitabine and 0
capecitabine trastuzumab 0
capecitabine cancer 0
capecitabine emtansine 0
capecitabine post-neoadjuvant 0
radiotherapy in 0
radiotherapy treatment 0
radiotherapy capecitabine 0
radiotherapy breast 0
radiotherapy with 0
radiotherapy and 0
radiotherapy trastuzumab 0
radiotherapy cancer 0
radiotherapy emtansine 0
radiotherapy post-neoadjuvant 0
radiochemotherapy in 0
radiochemotherapy treatment 0
radiochemotherapy capecitabine 0
radiochemotherapy breast 0
radiochemotherapy with 0
radiochemotherapy and 0
radiochemotherapy trastuzumab 0
radiochemotherapy cancer 0
radiochemotherapy emtansine 0
radiochemotherapy post-neoadjuvant 0
breast cancer in 0
breast cancer treatment 0
breast cancer capecitabine 0
breast cancer breast 1
breast cancer with 0
breast cancer and 0
breast cancer trastuzumab 0
breast cancer cancer 1
breast cancer emtansine 0
breast cancer post-ne

In [108]:
print(f'subs: {substring_match(authorkeys, manualkeys)}')

capecitabine post-neoadjuvant 0
capecitabine breast 0
capecitabine cancer 0
capecitabine trastuzumab 0
capecitabine capecitabine 1
capecitabine and 0
capecitabine in 1
capecitabine with 0
capecitabine emtansine 0
capecitabine treatment 0
breast cancer post-neoadjuvant 0
breast cancer breast 1
breast cancer cancer 1
breast cancer trastuzumab 0
breast cancer capecitabine 0
breast cancer and 0
breast cancer in 0
breast cancer with 0
breast cancer emtansine 0
breast cancer treatment 0
radiochemotherapy post-neoadjuvant 0
radiochemotherapy breast 0
radiochemotherapy cancer 0
radiochemotherapy trastuzumab 0
radiochemotherapy capecitabine 0
radiochemotherapy and 0
radiochemotherapy in 0
radiochemotherapy with 0
radiochemotherapy emtansine 0
radiochemotherapy treatment 0
radiotherapy post-neoadjuvant 0
radiotherapy breast 0
radiotherapy cancer 0
radiotherapy trastuzumab 0
radiotherapy capecitabine 0
radiotherapy and 0
radiotherapy in 0
radiotherapy with 0
radiotherapy emtansine 0
radiotherapy 